# Libraries

In [1]:
import numpy as np
import pandas as pd

In [2]:
from nltk.tokenize import word_tokenize
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical

Using TensorFlow backend.


In [3]:
from sklearn.naive_bayes import MultinomialNB

In [4]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [5]:
import pickle

In [6]:
pd.options.display.max_colwidth = -1

# Data

In [7]:
data = pd.read_csv('E:/Scrapped-Data/Quotes-Goodreads/Popular.csv')

In [8]:
data

,Quote
0,"“Don't cry because it's over, smile because it happened.” ― Dr. Seuss"
1,"“I'm selfish, impatient and a little insecure. I make mistakes, I am out of control and at times hard to handle. But if you can't handle me at my worst, then you sure as hell don't deserve me at my best.” ― Marilyn Monroe"
2,“Be yourself; everyone else is already taken.” ― Oscar Wilde
3,“Two things are infinite: the universe and human stupidity; and I'm not sure about the universe.” ― Albert Einstein
4,"“So many books, so little time.” ― Frank Zappa"
...,...
2995,“What can you do to promote world peace? Go home and love your family.” ― Mother Teresa
2996,"“. . . when it comes down to it, that’s what life is all about: showing up for the people you love, again and again, until you can’t show up anymore.” ― Rebecca Walker, Baby Love: Choosing Motherhood After a Lifetime of Ambivalence //"
2997,"“Just tell me how to be different in a way that makes sense.” ― Stephen Chbosky, The Perks of Being a Wallflower //"
2998,"“sometimes you don't need a goal in life, you don't need to know the big picture. you just need to know what you're going to do next!” ― Sophie Kinsella, The Undomestic Goddess //"


# Cleaning

In [9]:
def clean(x):
    x = x.split('―')[0]
    x = x[1:-2]
    return x+ ' endquote'

In [10]:
data['Quote'] = data['Quote'].apply(clean)

In [11]:
data

,Quote
0,"Don't cry because it's over, smile because it happened. endquote"
1,"I'm selfish, impatient and a little insecure. I make mistakes, I am out of control and at times hard to handle. But if you can't handle me at my worst, then you sure as hell don't deserve me at my best. endquote"
2,Be yourself; everyone else is already taken. endquote
3,Two things are infinite: the universe and human stupidity; and I'm not sure about the universe. endquote
4,"So many books, so little time. endquote"
...,...
2995,What can you do to promote world peace? Go home and love your family. endquote
2996,". . . when it comes down to it, that’s what life is all about: showing up for the people you love, again and again, until you can’t show up anymore. endquote"
2997,Just tell me how to be different in a way that makes sense. endquote
2998,"sometimes you don't need a goal in life, you don't need to know the big picture. you just need to know what you're going to do next! endquote"


# Preprocessing

In [12]:
tokens = word_tokenize('\n'.join(data['Quote']))

In [13]:
train_len = 4+1
text_sequences = []
for i in range(train_len, len(tokens)+1):
    seq = tokens[i-train_len:i]
    text_sequences.append(seq)    

In [14]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)

In [15]:
vocabulary_size = len(tokenizer.index_word)+1
print(vocabulary_size)

8539


In [16]:
sequences = tokenizer.texts_to_sequences(text_sequences)
sequences = np.array(sequences)

In [17]:
X = sequences[:, :-1]
y = sequences[:, -1]
# y = to_categorical(y, num_classes=vocabulary_size)

sequence_len = train_len-1

# Training

In [18]:
model = MultinomialNB()
# model.fit(X,y)
1/0

ZeroDivisionError: division by zero

In [19]:
model.fit(X, y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
model.score(X,y)

In [ ]:
es = EarlyStopping(monitor='loss', mode='min', verbose=1)
filepath = "model.h5"
ckpt = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')

def build_network():
    model = Sequential()
    model.add(Embedding(vocabulary_size, 128, input_length=sequence_len))
    model.add(LSTM(1024, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(vocabulary_size, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    return model

In [ ]:
model = build_network()

In [ ]:
model.fit(X, y, validation_split=0.3, epochs=500, callbacks=[es, ckpt])

from keras.models import load_model
new_model = load_model("model.h5")